# Imports and Pre-requisites

In [1]:
import json

import ee
import geemap
from geemap import geojson_to_ee, ee_to_geojson

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="rwanda-climate-alerts")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


# Pretty print a JSON file

In [7]:
sector_boundaries_path = "../data/boundaries/Sector_Boundaries.geojson"
district_boundaries_path = "../data/boundaries/District_Boundaries.geojson"

def pretty_print_json(file_path):
    with open(file_path, "r") as infile:
        data = json.load(infile)

    with open(file_path, "w") as outfile:
        json.dump(data, outfile, indent=None)

# pretty_print_json(sector_boundaries_path)
# pretty_print_json(district_boundaries_path)

# Import GeoJSONs

In [4]:
m = geemap.Map()

In [6]:
sector_boundaries_path = "../data/boundaries/Sector_Boundaries.geojson"
district_boundaries_path = "../data/boundaries/District_Boundaries.geojson"

with open(sector_boundaries_path, "r") as f:
    sector_boundaries = json.load(f)

with open(district_boundaries_path, "r") as f:
    district_boundaries = json.load(f)

print(type(sector_boundaries))

<class 'dict'>


In [8]:
sector_boundaries_ee = ee.FeatureCollection(sector_boundaries_path)
district_boundaries_ee = ee.FeatureCollection(district_boundaries_path)

# Note:
# ee.FeatureCollection() works with file paths

In [9]:
print(type(sector_boundaries_ee))

<class 'ee.featurecollection.FeatureCollection'>


In [ ]:
# Example: Get Bugesera District (district_id = 57)
bugesera = rwanda.filter(ee.Filter.eq("district_id", "57"))

# Or a specific sector (sector_id = 5704)
mareba = rwanda.filter(ee.Filter.eq("sector_id", "5704"))

# Extract the geometry
geometry = mareba.geometry()

# Print info
print("Mareba sector geometry:", geometry.getInfo())

# Example: calculate its area in square kilometers
area_km2 = geometry.area().divide(1e6).getInfo()
print("Mareba sector area (km²):", area_km2)

# Test GeoJson

In [ ]:
# Define a Geometry object.
geometry = ee.Geometry({
    'type': 'Polygon',
    'coordinates': [[
        [-122.081, 37.417],
        [-122.086, 37.421],
        [-122.084, 37.418],
        [-122.089, 37.416],
    ]],
})

# Apply the toGeoJSONString method to the Geometry object.
geometry_to_geojson_string = geometry.toGeoJSONString()

# Print the result.
display('geometry.toGeoJSONString(...) =', geometry_to_geojson_string)

# Display relevant geometries on the map.
m = geemap.Map()
m.set_center(-122.085, 37.422, 15)
m.add_layer(geometry, {'color': 'black'}, 'Geometry [black]: geometry')
m

# Collect Datasets

## Rainfall